<a href="https://colab.research.google.com/github/sssuinlee/BDP_project/blob/main/tensor_rnn_1210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
#csv 읽어오기
train = pd.read_csv("/content/drive/MyDrive/OTTO/train_prep.csv")
train = train.drop(train.columns[[0]],axis = 1)
display(train)

,customer_id,product_code,time_stamp,event_type
0,0,1517085,2022-07-31 22:00:00.025,clicks
1,0,1563459,2022-07-31 22:01:44.511,clicks
2,0,1309446,2022-08-01 15:23:59.426,clicks
3,0,16246,2022-08-01 15:28:39.997,clicks
4,0,1781822,2022-08-01 15:31:11.344,clicks
...,...,...,...,...
10285556,199999,928064,2022-08-01 06:46:48.967,clicks
10285557,199999,849970,2022-08-01 06:47:29.078,clicks
10285558,199999,1052480,2022-08-01 06:49:07.035,clicks
10285559,199999,487255,2022-08-01 06:49:21.116,clicks


In [3]:
#customer 0 의 세션
customer_0 = train[train.customer_id == 0]
display(customer_0)

,customer_id,product_code,time_stamp,event_type
0,0,1517085,2022-07-31 22:00:00.025,clicks
1,0,1563459,2022-07-31 22:01:44.511,clicks
2,0,1309446,2022-08-01 15:23:59.426,clicks
3,0,16246,2022-08-01 15:28:39.997,clicks
4,0,1781822,2022-08-01 15:31:11.344,clicks
...,...,...,...,...
271,0,843110,2022-08-28 10:58:18.768,clicks
272,0,938007,2022-08-28 10:59:15.390,clicks
273,0,1228848,2022-08-28 11:02:08.943,clicks
274,0,1740927,2022-08-28 11:09:02.173,clicks


In [4]:
#customer 0 의 orders 목록
customer_0_orders = customer_0[customer_0.event_type == 'orders']
display(customer_0_orders)

,customer_id,product_code,time_stamp,event_type
8,0,305831,2022-08-01 16:07:07.105,orders
9,0,461689,2022-08-01 16:07:07.105,orders
245,0,1199474,2022-08-26 22:29:00.651,orders
246,0,543308,2022-08-26 22:29:00.651,orders


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

In [6]:
#OneHotEncoder 객체에 전체 데이터의 pid랑 etype 넣어서 fit
ohe = OneHotEncoder(handle_unknown="ignore")
pid_etype = pd.DataFrame(data={"product_code":train.product_code, "event_type":train.event_type})
ohe.fit(pid_etype)

OneHotEncoder(handle_unknown='ignore')

In [7]:
#OneHotEncoding 객체 column확인:[1, 3, 4, ..., 1855600, 1855601, carts, clicks, orders]
ohe.categories_

[array([      1,       3,       4, ..., 1855598, 1855600, 1855601]),
 array(['carts', 'clicks', 'orders'], dtype=object)]

In [8]:
#pid_etype_0 = customer 0 의 pid랑 etype
pid_etype_0 = pd.DataFrame(data={"product_code":customer_0.product_code, "event_type":customer_0.event_type})
display(pid_etype_0[:-1])

,product_code,event_type
0,1517085,clicks
1,1563459,clicks
2,1309446,clicks
3,16246,clicks
4,1781822,clicks
...,...,...
270,219925,clicks
271,843110,clicks
272,938007,clicks
273,1228848,clicks


In [9]:
pid_etype_0[-1:]

,product_code,event_type
275,161938,clicks


In [10]:
#encoded_0 = customer0의 pid랑 etype 을 원핫인코딩
encoded_0 = ohe.transform(pid_etype_0).toarray()

In [11]:
encoded_0.shape

(276, 951297)

In [ ]:
#원핫인코딩 풀었을때 
type(ohe.inverse_transform(encoded_0))

In [ ]:
"""
input_batch=[]
input_batch.append(encoded_0)
input_batch
"""

[array([[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]])]

In [ ]:
#한 세션 = 한 문장
#pid 1개 = 한 단어
#session(sentence) = "15512 5532 85811 11390 1002 3 52"
#session: pid_etype_0 같은거("pid":(...), "etype":(...))
#sessions: pid_etype_0 같은것들의 'list'. (단, type(sessions)=list)

customer_0 = train[train.customer_id == 0]
pid_etype_0 = pd.DataFrame(data={"product_code":customer_0.product_code, "event_type":customer_0.event_type})

customer_1 = train[train.customer_id == 1]
pid_etype_1 = pd.DataFrame(data={"product_code":customer_1.product_code, "event_type":customer_1.event_type})

customer_2 = train[train.customer_id == 2]
pid_etype_2 = pd.DataFrame(data={"product_code":customer_2.product_code, "event_type":customer_2.event_type})

sessions = [pid_etype_0, pid_etype_1, pid_etype_2]
#type(sessions) = list

In [ ]:
batch_size = len(sessions)

def batchmaker(sessions):
  input_batch = []
  target_batch = []

  for session in sessions:  #한 세션씩
     input_batch.append(ohe.transform(session[:-1]).toarray())
     target_batch.append(session[-1:])

  return input_batch, target_batch

In [ ]:
input_batch, target_batch = batchmaker(sessions)

In [ ]:
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad = True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)

In [ ]:
dtype = torch.float

class SessionRNN(nn.Module):
  def __init__(self):
    super(SessionRNN, self).__init__()

    self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    self.Softmax = nn.Softmax(dim=1)

  def forward(self, hidden, X):
    X = X.transpose(0, 1)
    outputs, hidden = self.rnn(X, hidden)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    return model

In [ ]:
model = SessionRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(10):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model(hidden, input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [session.split()[:2] for session in sessions]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([session.split()[:2] for session in sessions], '->', [number_dict[n.item()] for n in predict.squeeze()])